In [1]:
import os
import pandas as pd
from lxml import html
import requests
import re
import sys
from datetime import datetime

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import time

In [2]:
cur_wd = str(os.getcwd())
cur_wd = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'data')
print(cur_wd)                                 

C:\Users\jjanko\Desktop\wei_mutual_fund_project\data


In [3]:
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]


In [4]:
weblink = {}


"""
CIK_LIST = ['867297', '1091439', '1301708', '1423227', '1098605', '1378240', '726735', '844779', '1398078', '774013',\
'1062806', '1026144', '1062805', '276463', '1097077', '1137391', '1160864', '1181249', '1159038', '353281', '834237',\
'97098', '216557', '765199', '814507', '790525', '110055', '1159040', '1137393', '1176194', '877151', '1169029', '1167467',\
'319108', '1051003', '1137440', '1167470', '35315', '722574', '225322', '795422', '35315', '803013', '729218', '205323',\
'1303459', '751199', '81205', '320351', '354046', '1401097', '35341', '754510', '35348', '275309', '819118', '880195',\
'1364924', '1061130', '744822', '278001', '719451', '225323', '880709', '708191', '917286', '61397', '356476', '43498',\
'1331971', '1329954', '1176199', '743861', '1189740', '1260041', '1215913', '22370', '1396502', '315554', '45291', '26262',\
'759866', '791271', '759828', '855886', '756913', '856671', '857769', '45288', '792858', '315554', '45288', '792858',\
'1496749', '22370', '925683', '45291', '759866', '791271', '741375', '1227155', '315812', '356409', '832705', '1161973',\
'836487', '730044', '716716', '860720', '93284', '806564', '916618', '93285', '924394', '1388141', '878929', '1368493',\
'904112', '859037', '919808', '876162', '1072552', '1132218', '1002427', '882381', '314366', '882381', '1074111', '889128',\
'100786', '881466', '316411', '817841', '750679', '726291', '750756', '1031594', '709146', '100802', '14358', '752372',\
'100852', '1238182', '813240', '793701', '857694', '1332239', '1109076', '1092633', '1142702', '862813', '1142702',\
'1086890', '1392577', '1238182', '1092633', '1086890', '1471420', '858744', '892538', '799084', '795307', '724129',\
'1059040', '1020861', '1274768', '1020861', '1327129', '1338170', '313212', '852254', '902259', '1177017', '1034386',\
'918294', '920467', '754915', '1120925', '1533490', '923084', '202927', '853437', '927845', '871839', '931151', '949820',\
'894024', '1019286', '1002624', '858581', '1181628', '1169187', '918292', '887147', '80248', '80249', '731890', '75170',\
'834798', '106444', '36405', '752177', '225997', '734383', '932471', '794105', '826473', '1273878', '857489', '1004655',\
'836906', '857490', '821404', '788599', '862341', '788606', '891190', '791107', '34066', '1070414', '105544', '105563',\
'68138', '52848', '799127', '736054', '106830', '313850', '107606', '806633', '806633']


"""

CIK_LIST = ['1681576', '1707770']


error = []
txts = []
for CIK in CIK_LIST:
    weblink[CIK] = []
    direc = os.path.join(cur_wd, CIK, 'n-q')
    if not os.path.exists(direc):
        os.makedirs(direc)
        
        req = Request(r"https://www.sec.gov/Archives/edgar/data/" + str(CIK))
        html_page = urlopen(req, timeout=10)

        soup = BeautifulSoup(html_page, "lxml")

        for link in soup.findAll('a'):
            archive = link.get('href')
            if str('/Archives/edgar/data/') + CIK in archive:
                try:
                    req_archive = Request(r"https://www.sec.gov" + archive)

                    html_page_archive = urlopen(req_archive, timeout=10)
                    soup_archive = BeautifulSoup(html_page_archive, "lxml")

                    for link_archive in soup_archive.findAll('a'):
                        txt_name = link_archive.get('href')
                        if ".txt" in txt_name:
                            ixs = find(txt_name, r'/')
                            file_name = txt_name[ixs[-1]+1:]
                            non_dig = re.findall(r'\D+', file_name)
                            if len(non_dig) == 3:
                                print(r"https://www.sec.gov" + txt_name)
                                try: 
                                    r = Request(r"https://www.sec.gov" + txt_name)
                                    read = urlopen(r, timeout=10)
                                    r = read.readlines()
                                    found = False
                                    #check to see if n-q file
                                    for i in r:
                                        string_row = i.decode('utf-8').lower()
                                        if 'conformed submission' in string_row and 'n-q'in string_row:
                                            found = True
                                except IncompleteRead:
                                    found = False

                                if found:
                                    print(r"https://www.sec.gov" + txt_name)
                                    weblink[CIK].append(r"https://www.sec.gov" + txt_name)
                                    ixs = find(txt_name, r'/')
                                    file_name = txt_name[ixs[-1]+1:]
                                    f = open(os.path.join(direc, file_name), mode = 'w')
                                    for row in r:
                                        clean_row = row.decode('utf-8')
                                        if clean_row != '':
                                            f.write(clean_row)
                                    f.close()
                except:
                    error.append(r"https://www.sec.gov" + archive)

https://www.sec.gov/Archives/edgar/data/1681576/000085343720000069/0000853437-20-000069.txt
https://www.sec.gov/Archives/edgar/data/1681576/000174177320002945/0001741773-20-002945.txt
https://www.sec.gov/Archives/edgar/data/1681576/000174177320002880/0001741773-20-002880.txt
https://www.sec.gov/Archives/edgar/data/1681576/000174177320002880/0001741773-20-002880.txt
https://www.sec.gov/Archives/edgar/data/1681576/999999999520002565/9999999995-20-002565.txt
https://www.sec.gov/Archives/edgar/data/1681576/000168157620000015/0001681576-20-000015.txt
https://www.sec.gov/Archives/edgar/data/1681576/000120677420002708/0001206774-20-002708.txt
https://www.sec.gov/Archives/edgar/data/1681576/000031696820000031/0000316968-20-000031.txt
https://www.sec.gov/Archives/edgar/data/1681576/000168157620000009/0001681576-20-000009.txt
https://www.sec.gov/Archives/edgar/data/1681576/000168157620000001/0001681576-20-000001.txt
https://www.sec.gov/Archives/edgar/data/1681576/000174177320002226/0001741773-20

https://www.sec.gov/Archives/edgar/data/1681576/000117701717000214/0001177017-17-000214.txt
https://www.sec.gov/Archives/edgar/data/1681576/000168157617000006/0001681576-17-000006.txt
https://www.sec.gov/Archives/edgar/data/1681576/000168157617000001/0001681576-17-000001.txt
https://www.sec.gov/Archives/edgar/data/1681576/000168157617000001/0001681576-17-000001.txt
https://www.sec.gov/Archives/edgar/data/1681576/000117701717000130/0001177017-17-000130.txt
https://www.sec.gov/Archives/edgar/data/1681576/000120677417002659/0001206774-17-002659.txt
https://www.sec.gov/Archives/edgar/data/1681576/000031696817000038/0000316968-17-000038.txt
https://www.sec.gov/Archives/edgar/data/1681576/000117701717000116/0001177017-17-000116.txt
https://www.sec.gov/Archives/edgar/data/1681576/000077978517000035/0000779785-17-000035.txt
https://www.sec.gov/Archives/edgar/data/1681576/000120677417002090/0001206774-17-002090.txt
https://www.sec.gov/Archives/edgar/data/1681576/000094982017000007/0000949820-17

https://www.sec.gov/Archives/edgar/data/1707770/000087183918000070/0000871839-18-000070.txt
https://www.sec.gov/Archives/edgar/data/1707770/000087183918000068/0000871839-18-000068.txt
https://www.sec.gov/Archives/edgar/data/1707770/000087183918000067/0000871839-18-000067.txt
https://www.sec.gov/Archives/edgar/data/1707770/000087183918000061/0000871839-18-000061.txt
https://www.sec.gov/Archives/edgar/data/1707770/000157176818000020/0001571768-18-000020.txt
https://www.sec.gov/Archives/edgar/data/1707770/000087183918000017/0000871839-18-000017.txt
https://www.sec.gov/Archives/edgar/data/1707770/000087183918000005/0000871839-18-000005.txt
https://www.sec.gov/Archives/edgar/data/1707770/000087183918000004/0000871839-18-000004.txt
https://www.sec.gov/Archives/edgar/data/1707770/000170777018000010/0001707770-18-000010.txt
https://www.sec.gov/Archives/edgar/data/1707770/000170777018000010/0001707770-18-000010.txt
https://www.sec.gov/Archives/edgar/data/1707770/000157176818000013/0001571768-18

In [5]:
error

['https://www.sec.gov/Archives/edgar/data/1707770/000120677419003006']